In [3]:
import logging
import ecomis
import pandas as pd
import numpy as np
import posixpath
from datetime import datetime
import time
import xlwings as xw
import copy

logger = logging.getLogger("juplogger")
handler = ecomis.LogViewver()
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

handler.show_logs()

Output(layout=Layout(border='solid', height='600px', width='100%'))

**Ingresos Nuevos**
----------
Ejecutar el siguiente codigo como requisito para las siguientes secciones de Ingresos Nuevos

In [4]:
month="202005"
inifile = ecomis.ReadIniFile(mercado="empresas")
parser = inifile.getIniFileParser()

# Configurando los archivos de configuración para administrar
# la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = ecomis.DbDataProcess(month)
dbmanager.setParser(dbparser)

***<span style="color:red">Copiar</span>*** Ingresos Nuevos del Archivo de Atracción
-----

Copia los ingresos nuevos desde el archivo enviado por atracción hacia el archivo de Ingresos Nuevos de comisiones.

**Requisitos**: 
- Haber filtrado las personas de interes en el archivo que envia attracion
- Asegurarse la hoja se llame BASE en el archivo de ingresos nuevos de comisiones
- Asegurarse que los nombres de columnas del archivo de atraccion no hayan cambiado

**Pasos**:
- En el archivo de atracción filtrar solo las personas que comisionan, eliminar las demás
- Ejecutar script
- Convertir la columna DNI a formato Texto
- Jalar las fórmulas
- Proseguir con la subida a la base de datos de los nuevos ingresos





In [4]:
## Cargamos el archivo de Ingresos (RRHH Ingresos.xlsx) porque para que 
# xlwings abra el archivo necesita el nombre del fichero y también
# necesitamos las columnas definidas en el archivo de configuracion.
section_ingresos = ecomis.SectionObj(inifile,"Ingresos2")
# Guardamos el nombre de nuestro archivo de nuevos ingresos.
comis_ingresos_archivo_nombre = section_ingresos.getParameter('filelist')[0]
# Obtenemos las columnas que deberia tener el reporte de ingresos nuevos.
cols_required = section_ingresos.getParameter('cols')


## Obtenemos la lista de los nombres de los libros
# actualmente abiertos en Excel.
wb_names = ecomis.getOpenedWorkbooksNames()
#Seleccionamos el archivo que contenga "REPORTEREMUNERACIONES"
keyname = "REPORTEREMUNERACIONES"
atraccion_ingresos_filename = [s for s in wb_names if keyname in s][0]
#Nos conectamos al archivo y hoja de calculo enviado por atraccion
wb_ingresos = xw.Book(atraccion_ingresos_filename)
sht = wb_ingresos.sheets[0]
#Nos posicionamos en la celda A1 extendemos hacia la esquina opuesta 
#retornando el contenido como dataframe
df = sht.range('A1').options(pd.DataFrame,
                         index=False, expand='table').value

#Filtramos las columnas que debe tener el archivo de atraccion.
#Eliminamos los 3 ultimos elementos de cols_required y concatenamos
#con las demas columnas que contienen informacion que necesitamos.
cols_required = cols_required[:-3] + ['Variable', 'VP', 'CC', 'Jefe DIrecto']
logger.info(cols_required[:-3])
#Obtenemos el dataframe a ser pegado en nuestro archivo de comisiones.
df_final = df[cols_required]

#Nos conectamos a nuestro archivo de Ingresos
wb_comis_ingresos = xw.Book(comis_ingresos_archivo_nombre)
sheet_comis_ingresos = wb_comis_ingresos.sheets('BASE')
#Obtener el nro. de fila del último registro
lastrow = sheet_comis_ingresos.api.Cells(65536, 1).End(xw.constants.Direction.xlUp).Row
#Pegamos el dataframe desde la primera fila vacia hacia abajo.
sheet_comis_ingresos.range((lastrow+1, 1)).options(header=False, index=False).value = df_final



Output(layout=Layout(border='solid', height='250px', width='100%'), outputs=({'name': 'stdout', 'output_type':…

Insertar ***<span style="color:red">Nuevos</span>*** Ingresos en la BD
-----

Inserta los registros mracados con ADD_TO_DB = "SI" del dataframe de Ingresos de comisiones a la base de datos

In [6]:
section_ingresos = ecomis.SectionObj(inifile,"Ingresos")
ingresos = ecomis.LoadFileProcess(section_ingresos).loadFile()
section_ingresos_cdp = ecomis.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = ecomis.LoadFileProcess(section_ingresos_cdp).loadFile()

# Separamos los que se van a agregar a la base de datos y los que unicamente se va actualizar su CODIGO_INAR
ingresos_add = ingresos[ingresos.ADD_TO_DB == "SI"]
ingresos_add.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_TARGET'], axis=1, inplace=True)

ingresos_add_CDP = ingresos_cdp[ingresos_cdp.ADD_TO_DB == "SI"]
ingresos_add_CDP.drop(['ADD_TO_DB','UPDATE_PUESTO','UPDATE_CODIGO_INAR','UPDATE_TARGET'], axis=1, inplace=True)

C:\Users\jsaenza\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
ingresos_add

,DNI,NOMBRES,APELLIDO_PATERNO,POSICION_EMPL,FECHA_INGRESO,CODIGO_INAR,TARGET
4,47451980,CONDORI PICON MIGUEL LINO,CONDORI PICON,CONSULTOR VENTAS DIRECTAS,2020-04-01,mlcondori,3000
5,09999439,NÚÑEZ SALAZAR KARINNA JANET,NÚÑEZ SALAZAR,EJECUTIVO INSIDE SALES,2020-04-01,kjnuñez,1800


**Ejecutar lo siguiente para modificar la BD**

In [8]:
dbmanager.dbOperation('insertwodeletion','tblEmpleados',ingresos_add)
#dbmanager.dbOperation('insertwodeletion','tblEmpleados',ingresos_add_CDP)

Actualizar ***<span style="color:red">Codigo Inar</span>*** Ingresos en la BD
----

Actualiza los registros marcados con UPDATE_CORDIGO_INAR = "SI" del dataframe de ingresos a la Base de Datos.

In [11]:
section_ingresos = ecomis.SectionObj(inifile,"Ingresos")
ingresos = ecomis.LoadFileProcess(section_ingresos).loadFile()
section_ingresos_cdp = ecomis.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = ecomis.LoadFileProcess(section_ingresos_cdp).loadFile()

ingresos_update = ingresos[ingresos.UPDATE_CODIGO_INAR == "SI"]
ingresos_update.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_TARGET'], axis=1, inplace=True)

ingresos_update_CDP = ingresos_cdp[ingresos_cdp.UPDATE_CODIGO_INAR == "SI"]
ingresos_update_CDP.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_PUESTO','UPDATE_TARGET'], axis=1, inplace=True)
ingresos_update_CDP

,DNI,NOMBRES,POSICION_EMPL,FECHA_INGRESO,TARGET


In [14]:
ingresos_update

,DNI,NOMBRES,APELLIDO_PATERNO,POSICION_EMPL,FECHA_INGRESO,CODIGO_INAR,TARGET


**Ejecutar lo siguiente para modificar la BD**

In [13]:
dbmanager.dbOperation('update','UpdateEmpleadoCodigoInar',ingresos_update)
dbmanager.dbOperation('update','UpdateEmpleadoCodigoInar',ingresos_update_CDP)

Actualizar ***<span style="color:red">Puestos</span>*** Ingresos en la BD
-----

Actualiza los puestos por DNI ingresados en el archivo de Ingresos de comisiones pestaña "UpdatePuestos" hacia la Base de Datos.

In [3]:
section_ingresos_cdp = ecomis.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = ecomis.LoadFileProcess(section_ingresos_cdp).loadFile()

ingresos_update_puesto_CDP = ingresos_cdp[ingresos_cdp.UPDATE_PUESTO == "SI"]
ingresos_update_puesto_CDP.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_PUESTO','UPDATE_TARGET'], axis=1, inplace=True)

In [4]:
ingresos_update_puesto_CDP

,DNI,NOMBRES,POSICION_EMPL,FECHA_INGRESO,TARGET
0,45219594,TAPIA PANDURO KEY ELAR,CONSULTOR SENIOR VENTAS DIRECTAS,2019-11-01,4200


**Ejecutar lo siguiente para modificar la BD**

In [5]:
dbmanager.dbOperation('update','UpdatePuestoEmpleado',ingresos_update_puesto_CDP)

Actualizar ***<span style="color:red">TARGETS</span>*** Ingresos en la BD
-----

Actualiza los targets por DNI en la base de datos.

In [7]:
section_ingresos = ecomis.SectionObj(inifile,"Ingresos")
ingresos = ecomis.LoadFileProcess(section_ingresos).loadFile()
section_ingresos_cdp = ecomis.SectionObj(inifile,"IngresosCDP")
ingresos_cdp = ecomis.LoadFileProcess(section_ingresos_cdp).loadFile()

ingresos_update = ingresos[ingresos.UPDATE_TARGET == "SI"]
ingresos_update.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_TARGET'], axis=1, inplace=True)

ingresos_update_CDP = ingresos_cdp[ingresos_cdp.UPDATE_TARGET == "SI"]
ingresos_update_CDP.drop(['ADD_TO_DB','UPDATE_CODIGO_INAR','UPDATE_PUESTO','UPDATE_TARGET'], axis=1, inplace=True)
ingresos_update_CDP

C:\Users\jsaenza\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,DNI,NOMBRES,POSICION_EMPL,FECHA_INGRESO,TARGET


In [8]:
ingresos_update

,DNI,NOMBRES,APELLIDO_PATERNO,POSICION_EMPL,FECHA_INGRESO,CODIGO_INAR,TARGET
12,41215255,ALFARO LUNA VICTORIA CARLOS ALBERTO,ALFARO LUNA VICTORIA,EJECUTIVO DE DESARROLLO PYME REGIONES,2019-09-01,calfaro,2200
13,71407612,ARIMBORGO DIAZ ROSARIO,ARIMBORGO DIAZ,ASESOR DE SERVICIOS EMPRESAS PLUS,2019-09-01,rarimborgo,600
15,43166770,CONTRERAS SANCHEZ JOSE FRANCISCO,CONTRERAS SANCHEZ,EJECUTIVO DE DESARROLLO PYME REGIONES,2019-09-01,jcontreras,2200
17,40969083,LAZARO RUSSELL CHRISTIAN ANTHONY,LAZARO RUSSELL,EJECUTIVO DE DESARROLLO PYME REGIONES,2019-09-01,clazaro,2200
19,48272168,PAREDES VEGA ESTHER CELINA,PAREDES VEGA,ASESOR DE SERVICIOS EMPRESAS PLUS,2019-09-01,eparedes,600
20,44872218,SANTA CRUZ BAZALAR JOSE CARLOS,SANTA CRUZ BAZALAR,EJECUTIVO DE DESARROLLO PYME,2019-09-01,jsantacruz,2500
21,48153838,TELLO TELLO ALICIA,TELLO TELLO,EJECUTIVO DE DESARROLLO PYME REGIONES,2019-09-01,atello,2200
23,41663955,ZEBALLOS RIVAS ANDREA,ZEBALLOS RIVAS,EJECUTIVO DE REGIONES,2019-09-01,azeballos,4000
24,47145757,CHIRINOS ESPINOZA LUIS OMAR,CHIRINOS ESPINOZA,ASESOR DE SERVICIOS PREFERENTES,2019-09-16,lchirinos,600
26,47153042,MENDOZA ARÉVALO MARITA DEL CARMEN,MENDOZA ARÉVALO,EJECUTIVO DE DESARROLLO PYME,2019-09-16,mmendoza,2500


In [9]:
dbmanager.dbOperation('update','UpdateTargetEmpleado',ingresos_update)
dbmanager.dbOperation('update','UpdateTargetEmpleado',ingresos_update_CDP)